In [ ]:
# Cell 1: Install libraries and login
!pip install --upgrade transformers accelerate bitsandbytes flask pyngrok flask-cors markdown --quiet
!pip install huggingface_hub

from huggingface_hub import login
# Using your provided token
login("YOUR HUGGING FACE TOKEN")

In [2]:
# Cell 2: Load the model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
import torch
import markdown
import os

app = Flask(__name__)
CORS(app)

def generate_roadmap(data, model, tokenizer):
    exp_text = f"with {data.get('years_exp')} years of experience" if data.get('years_exp') else "as a fresher"

    # Strictly instructing the model to use the tags we are looking for
    system_instruction = (
        "You are a Senior Career Strategist. Talk directly TO the user using 'you' and 'your'. "
        "For each of the 3 career paths, give Career Match Score %. "
        "Provide a 30/60/90 day roadmap and skill gaps. "
        "VERY IMPORTANT: You MUST end your response with a final motivational paragraph wrapped in tags like this: "
        "[MOTIVATION] your motivational text here [/MOTIVATION]"
    )

    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": f"Analyze profile: {data['category']}, {exp_text}, Education: {data['edu']}, Skills: {data['skills']}, Interests: {data['interests']}, Goals: {data['goals']}"}
    ]

    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=1500,
            temperature=0.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    raw_response = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)

    main_content = raw_response
    motivation = ""

    # Logic to extract motivation between tags
    if "[MOTIVATION]" in raw_response:
        parts = raw_response.split("[MOTIVATION]", 1)
        main_content = parts[0]
        motivation = parts[1].split("[/MOTIVATION]")[0].strip()
    else:
        # Fallback: If AI forgets tags, take the last paragraph as motivation
        paragraphs = raw_response.strip().split('\n\n')
        if len(paragraphs) > 1:
            motivation = paragraphs[-1]
            main_content = '\n\n'.join(paragraphs[:-1])

    return markdown.markdown(main_content), motivation

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/recommend", methods=["POST"])
def recommend():
    from __main__ import model, tokenizer
    data = request.form
    html_report, motivation = generate_roadmap(data, model, tokenizer)
    return jsonify({"report": html_report, "motivation": motivation})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)

Overwriting app.py


In [4]:
# Cell 4: Create template
!mkdir -p templates


In [5]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Career Strategist | Llama 3.1</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;600;700&display=swap" rel="stylesheet">
    <style>
        :root { --primary-gradient: linear-gradient(135deg, #6366f1 0%, #a855f7 100%); --glass-bg: rgba(255, 255, 255, 0.95); --dark-bg: #0f172a; }
        body { background-color: var(--dark-bg); background-image: radial-gradient(circle at 0% 0%, rgba(99, 102, 241, 0.15) 0%, transparent 50%), radial-gradient(circle at 100% 100%, rgba(168, 85, 247, 0.15) 0%, transparent 50%); font-family: 'Inter', sans-serif; color: #f8fafc; min-height: 100vh; }
        .glass-card { background: var(--glass-bg); backdrop-filter: blur(12px); border: 1px solid rgba(255, 255, 255, 0.2); border-radius: 24px; color: #1e293b; box-shadow: 0 25px 50px -12px rgba(0, 0, 0, 0.5); overflow: hidden; }
        .form-label { font-weight: 600; color: #475569; font-size: 0.9rem; }
        .form-control, .form-select { border-radius: 12px; padding: 12px 16px; border: 1px solid #e2e8f0; background: #f8fafc; transition: all 0.3s ease; }
        .btn-premium { background: var(--primary-gradient); border: none; border-radius: 12px; padding: 14px; font-weight: 700; color: white; transition: transform 0.2s ease; }
        .btn-premium:hover { transform: translateY(-2px); box-shadow: 0 10px 20px -5px rgba(99, 102, 241, 0.5); }

        .motivation-banner {
            background: linear-gradient(to right, #f0f9ff, #e0f2fe);
            border-left: 6px solid #0ea5e9;
            border-radius: 16px;
            padding: 24px;
            color: #0369a1 !important;
            font-size: 1.1rem;
            font-weight: 500;
            line-height: 1.6;
            display: none;
            margin-top: 2rem;
            text-shadow: none;
        }

        .loading-overlay { display: none; position: absolute; top: 0; left: 0; width: 100%; height: 100%; background: rgba(255, 255, 255, 0.9); z-index: 10; flex-direction: column; align-items: center; justify-content: center; }

        #report { color: #1e293b; }
        #report h1, #report h2, #report h3 { color: #0f172a; margin-top: 1.5rem; font-weight: 700; }
    </style>
</head>
<body class="container py-5">
    <div class="row justify-content-center">
        <div class="col-lg-8">
            <div class="text-center mb-5">
                <h1 class="display-4 fw-bold text-white mb-2">AI Career <span style="background: var(--primary-gradient); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">Strategist</span></h1>
                <p class="text-light opacity-75">Powered by Meta Llama 3.1 • Personalized for Your Future</p>
            </div>
            <div class="glass-card p-4 p-md-5 position-relative">
                <div class="loading-overlay" id="loader">
                    <div class="spinner-border text-primary mb-3" style="width: 3rem; height: 3rem;"></div>
                    <h5 class="fw-bold text-dark">Designing something great for you...✨</h5>
                </div>
                <form id="careerForm">
                    <div class="row g-4">
                        <div class="col-md-6">
                            <label class="form-label">CURRENT CAREER STAGE</label>
                            <select name="category" id="catSelect" class="form-select" onchange="toggleExp()" required>
                                <option value="Entry-Level (Student / Fresher)">Entry-Level / Fresher</option>
                                <option value="Working Professional (1–3 years experience)">Working Professional (1-3 years)</option>
                                <option value="Experienced Professional">Experienced Professional (3+ years)</option>
                                <option value="Career Switcher">Career Switcher</option>
                                <option value="Returning After Career Break">Returning After Career Break</option>
                            </select>
                        </div>
                        <div class="col-md-6" id="expDiv" style="display:none;">
                            <label class="form-label">YEARS OF EXPERIENCE</label>
                            <input type="number" name="years_exp" class="form-control" placeholder="e.g. 5">
                        </div>
                        <div class="col-12"><label class="form-label">ACADEMIC BACKGROUND</label><input type="text" name="edu" class="form-control" placeholder="Degree, Major, University" required></div>
                        <div class="col-12"><label class="form-label">YOUR TECHNICAL SKILLS</label><textarea name="skills" class="form-control" rows="2" placeholder="SQL, Python, C#..." required></textarea></div>
                        <div class="col-md-6"><label class="form-label">CORE INTERESTS</label><input type="text" name="interests" class="form-control" placeholder="Microservices, UI/UX..." required></div>
                        <div class="col-md-6"><label class="form-label">LONG-TERM GOALS</label><input type="text" name="goals" class="form-control" placeholder="Senior Architect, CTO..." required></div>
                    </div>
                    <button type="submit" class="btn btn-premium w-100 mt-5 shadow-sm">GENERATE MY CAREER ROADMAP</button>
                </form>

                <div id="results" class="mt-5 pt-4 border-top" style="display:none;">
                    <div id="report"></div>
                    <div id="motivation-box" class="motivation-banner"></div>
                </div>
            </div>
        </div>
    </div>

    <script>
        function toggleExp() {
            const val = document.getElementById("catSelect").value;
            const showExp = val.includes("Working Professional") || val.includes("Experienced") || val.includes("Switcher");
            document.getElementById("expDiv").style.display = showExp ? "block" : "none";
        }

        document.getElementById("careerForm").onsubmit = async (e) => {
            e.preventDefault();
            const loader = document.getElementById("loader");
            const results = document.getElementById("results");
            const reportDiv = document.getElementById("report");
            const motivationBox = document.getElementById("motivation-box");

            loader.style.display = "flex";
            results.style.display = "none";
            motivationBox.style.display = "none";

            try {
                const response = await fetch("/recommend", { method: "POST", body: new FormData(e.target) });
                const data = await response.json();

                loader.style.display = "none";
                results.style.display = "block";

                // Show the main report
                reportDiv.innerHTML = data.report;

                // Show motivation in the box if it exists
                if (data.motivation && data.motivation.trim() !== "") {
                    motivationBox.innerHTML = `
                        <div style="display: flex; align-items: flex-start;">
                            <span style="font-size: 1.6rem; margin-right: 15px;">✨</span>
                            <div style="font-style: italic;">${data.motivation}</div>
                        </div>
                    `;
                    motivationBox.style.display = "block";
                }

                results.scrollIntoView({ behavior: 'smooth' });

            } catch (err) {
                console.error(err);
                alert("Connection error. Check your server.");
                loader.style.display = "none";
            }
        };
    </script>
</body>
</html>

Overwriting templates/index.html


In [ ]:
# Cell 5: High-Reliability Deployment Script
import threading
import time
import socket
from pyngrok import ngrok, conf

# --- 1. Aggressive Port & Process Cleanup ---
print("Stopping previous instances...")
!fuser -k 8000/tcp || true
!pkill -9 -f ngrok || true
time.sleep(3) # Wait for OS to release the socket

# --- 2. ngrok Configuration ---
# Using your provided auth token
conf.get_default().auth_token = "YOUR NGROK AUTH TOKEN"

# Clear existing tunnels to avoid session limit errors
try:
    for t in ngrok.get_tunnels():
        ngrok.disconnect(t.public_url)
except:
    pass

# Connect new tunnel
public_url = ngrok.connect(8000).public_url
print(f"\n🌍 PCPR PROJECT IS LIVE AT: {public_url}")

# --- 3. Stable Flask Runner ---
from app import app

def run_flask():
    # threaded=True and use_reloader=False are vital for Colab stability
    app.run(host="0.0.0.0", port=8000, use_reloader=False, threaded=True)

# Launch as a background thread
print("Starting Flask backend...")
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# --- 4. Final Verification ---
time.sleep(2)
print("✅ Ready! Use the Public URL above to generate your roadmaps.")

Stopping previous instances...
^C

🌍 PCPR PROJECT IS LIVE AT: https://catechumenal-pseudoapologetic-shelby.ngrok-free.dev
Starting Flask backend...
 * Serving Flask app 'app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit


✅ Ready! Use the Public URL above to generate your roadmaps.
